In [1]:
# default_exp models.wnbeats.nbeats

In [2]:
#hide
%load_ext autoreload
%autoreload 2

In [3]:
#export
import os
import time
import numpy as np
import pandas as pd
import random
from collections import defaultdict
import copy

import torch as t
from torch import optim
from pathlib import Path
from functools import partial

from nixtla.models.wnbeats.nbeats_model import NBeats, NBeatsBlock, IdentityBasis, TrendBasis, SeasonalityBasis
from nixtla.models.wnbeats.nbeats_model import XBasisTCN, XBasisWavenet
from nixtla.losses.pytorch import MAPELoss, MASELoss, SMAPELoss, MSELoss, MAELoss, RMSELoss
from nixtla.losses.numpy import mae, mse, mape, smape, rmse

In [4]:
#export
def init_weights(module, initialization):
    if type(module) == t.nn.Linear:
        if initialization == 'Orthogonal':
            t.nn.init.orthogonal_(module.weight)
        elif initialization == 'he_uniform':
            t.nn.init.kaiming_uniform_(module.weight)
        elif initialization == 'he_normal':
            t.nn.init.kaiming_normal_(module.weight)
        elif initialization == 'glorot_uniform':
            t.nn.init.xavier_uniform_(module.weight)
        elif initialization == 'glorot_normal':
            t.nn.init.xavier_normal_(module.weight)
        elif initialization == 'lecun_normal':
            t.nn.init.normal_(module.weight, 0.0, std=1/np.sqrt(module.weight.numel()))
        else:
            assert 1<0, f'Initialization {initialization} not found'

class Nbeats(object):
    """
    Future documentation
    """
    SEASONALITY_BLOCK = 'seasonality'
    TREND_BLOCK = 'trend'
    IDENTITY_BLOCK = 'identity'

    def __init__(self,
                 n_x_t,
                 n_x_s,
                 t_cols,
                 include_var_dict,
                 input_size_multiplier,
                 output_size,
                 shared_weights,
                 activation,
                 initialization,
                 stack_types,
                 n_blocks,
                 n_theta_hidden_list,
                 n_xbasis_layers,
                 n_xbasis_channels,
                 n_s_hidden,
                 scaler,
                 batch_normalization,
                 learning_rate,
                 lr_decay,
                 n_lr_decay_steps,
                 dropout_prob_theta,
                 dropout_prob_xbasis,
                 lambda_l1_theta,
                 lambda_l1_input,
                 lambda_l2_xbasis,
                 max_epochs,
                 early_stopping,
                 loss,
                 frequency,
                 seasonality,
                 random_seed,
                 device=None):
        super(Nbeats, self).__init__()

        if activation == 'selu': initialization = 'lecun_normal'

        # Attributes of ts_dataset
        # TODO: pensar como quitar redundancia
        self.n_x_t  = n_x_t
        self.n_x_s  = n_x_s
        self.t_cols = t_cols
        self.include_var_dict = include_var_dict

        # Architecture parameters
        self.input_size            = int(input_size_multiplier*output_size)
        self.output_size           = output_size
        self.shared_weights        = shared_weights
        self.activation            = activation
        self.initialization        = initialization
        self.stack_types           = stack_types
        self.n_blocks              = n_blocks
        self.n_theta_hidden_list   = n_theta_hidden_list
        self.n_xbasis_layers       = n_xbasis_layers
        self.n_xbasis_channels     = n_xbasis_channels
        self.n_s_hidden            = n_s_hidden

        # Regularization and optimization parameters
        self.scaler                = scaler
        self.batch_normalization   = batch_normalization
        self.learning_rate         = learning_rate
        self.lr_decay              = lr_decay
        self.n_lr_decay_steps      = n_lr_decay_steps
        # self.weight_decay         = weight_decay

        self.loss                  = loss
        self.dropout_prob_theta    = dropout_prob_theta
        self.dropout_prob_xbasis   = dropout_prob_xbasis
        self.lambda_l1_theta       = lambda_l1_theta
        self.lambda_l1_input       = lambda_l1_input
        self.lambda_l2_xbasis      = lambda_l2_xbasis
        self.max_epochs            = max_epochs
        self.early_stopping        = early_stopping

        # Regularization and optimization parameters
        self.frequency   = frequency
        self.seasonality = seasonality
        self.random_seed = random_seed

        if device is None: device = 'cuda' if t.cuda.is_available() else 'cpu'
        self.device = device
        self._is_instantiated = False

    def create_stack(self):
        if self.include_var_dict is not None:
            x_t_n_inputs = self.output_size * int(sum([len(x) for x in self.include_var_dict.values()]))

            # Correction because week_day only adds 1 no output_size
            if len(self.include_var_dict['week_day'])>0:
                x_t_n_inputs = x_t_n_inputs - self.output_size + 1
        else:
            x_t_n_inputs = self.input_size

        # Architecture definition
        block_list = []
        for i in range(len(self.stack_types)):
            #print(f'| --  Stack {self.stack_types[i]} (#{i})')
            for block_id in range(self.n_blocks[i]):
                # Batch norm only on first block
                if (len(block_list)==0) and (self.batch_normalization):
                    batch_normalization_block = True
                else:
                    batch_normalization_block = False

                # Shared weights
                if self.shared_weights and block_id>0:
                    nbeats_block = block_list[-1]

                else:
                    if self.stack_types[i] == 'identity':
                        nbeats_block = NBeatsBlock(x_t_n_inputs = x_t_n_inputs,
                                                   x_s_n_inputs = self.n_x_s,
                                                   x_s_n_hidden= self.n_s_hidden,
                                                   theta_n_dim=self.input_size + self.output_size,
                                                   basis=IdentityBasis(backcast_size=self.input_size,
                                                                       forecast_size=self.output_size),
                                                   n_theta_hidden_list=self.n_theta_hidden_list[i],
                                                   include_var_dict=self.include_var_dict,
                                                   t_cols=self.t_cols,
                                                   batch_normalization=batch_normalization_block,
                                                   dropout_prob=self.dropout_prob_theta,
                                                   activation=self.activation)
                    elif self.stack_types[i] == 'exogenous_tcn':
                        #assert len(self.f_cols)>0, 'If Xbasis, provide f_cols hyperparameter'
                        nbeats_block = NBeatsBlock(x_t_n_inputs = x_t_n_inputs,
                                                   x_s_n_inputs = self.n_x_s,
                                                   x_s_n_hidden = self.n_s_hidden,
                                                   theta_n_dim = 2*(self.n_xbasis_channels),
                                                   basis=XBasisTCN(out_features=self.n_xbasis_channels,
                                                                   in_features=self.n_x_t,
                                                                   #f_idxs=self.f_idxs,
                                                                   num_levels=self.n_xbasis_layers,
                                                                   dropout_prob=self.dropout_prob_xbasis),
                                                   n_theta_hidden_list=self.n_theta_hidden_list[i],
                                                   include_var_dict=self.include_var_dict,
                                                   t_cols=self.t_cols,
                                                   batch_normalization=batch_normalization_block,
                                                   dropout_prob=self.dropout_prob_theta,
                                                   activation=self.activation)
                    elif self.stack_types[i] == 'exogenous_wavenet':
                        #assert len(self.f_cols)>0, 'If Xbasis, provide f_cols hyperparameter'
                        nbeats_block = NBeatsBlock(x_t_n_inputs = x_t_n_inputs,
                                                   x_s_n_inputs = self.n_x_s,
                                                   x_s_n_hidden = self.n_s_hidden,
                                                   theta_n_dim = 2*(self.n_xbasis_channels),
                                                   basis=XBasisWavenet(out_features=self.n_xbasis_channels,
                                                                       in_features=self.n_x_t,
                                                                       #f_idxs=self.f_idxs,
                                                                       num_levels=self.n_xbasis_layers,
                                                                       dropout_prob=self.dropout_prob_xbasis),
                                                   n_theta_hidden_list=self.n_theta_hidden_list[i],
                                                   include_var_dict=self.include_var_dict,
                                                   t_cols=self.t_cols,
                                                   batch_normalization=batch_normalization_block,
                                                   dropout_prob=self.dropout_prob_theta,
                                                   activation=self.activation)

                # Select type of evaluation and apply it to all layers of block
                init_function = partial(init_weights, initialization=self.initialization)
                nbeats_block.layers.apply(init_function)
                block_list.append(nbeats_block)
        return block_list

    def __loss_fn(self, loss_name: str):
        def loss(x, freq, forecast, target, mask):
            if loss_name == 'MAPE':
                return MAPELoss(y=target, y_hat=forecast, mask=mask) + \
                       self.reg_loss_theta() + self.reg_loss_xbasis()
            elif loss_name == 'MASE':
                return MASELoss(y=target, y_hat=forecast, y_insample=x, seasonality=freq, mask=mask) + \
                       self.reg_loss_theta() + self.reg_loss_xbasis()
            elif loss_name == 'SMAPE':
                return SMAPELoss(y=target, y_hat=forecast, mask=mask) + self.l1_regularization() + \
                       self.reg_loss_theta() + self.reg_loss_xbasis()
            elif loss_name == 'MSE':
                return MSELoss(y=target, y_hat=forecast, mask=mask) + \
                       self.reg_loss_theta() + self.reg_loss_xbasis()
            elif loss_name == 'RMSE':
                return RMSELoss(y=target, y_hat=forecast, mask=mask) + \
                       self.reg_loss_theta() + self.reg_loss_xbasis()
            elif loss_name == 'MAE':
                return MAELoss(y=target, y_hat=forecast, mask=mask) + \
                       self.reg_loss_theta() + self.reg_loss_xbasis()
            else:
                raise Exception(f'Unknown loss function: {loss_name}')
        return loss

    def __val_loss_fn(self, loss_name: str):
        #TODO: mase not implemented
        def loss(forecast, target, weights):
            if loss_name == 'MAPE':
                return mape(y=target, y_hat=forecast, weights=weights) #TODO: faltan weights
            elif loss_name == 'SMAPE':
                return smape(y=target, y_hat=forecast, weights=weights) #TODO: faltan weights
            elif loss_name == 'MSE':
                return mse(y=target, y_hat=forecast, weights=weights)
            elif loss_name == 'RMSE':
                return rmse(y=target, y_hat=forecast, weights=weights)
            elif loss_name == 'MAE':
                return mae(y=target, y_hat=forecast, weights=weights)
            else:
                raise Exception(f'Unknown loss function: {loss_name}')
        return loss

    def reg_loss_xbasis(self):
        loss = 0.0
        for i in range(len(self.stack_types)):
            for block_id in range(self.n_blocks[i]):
                if self.stack_types[i] == 'exogenous_tcn':
                    for layer in self.model.blocks[i].modules():
                        if isinstance(layer, t.nn.Conv1d):
                            loss += self.lambda_l2_xbasis * t.norm(layer.weight)
        return loss

    def reg_loss_theta(self):
        # L1 loss for initial exogenous input
        loss = self.lambda_l1_input * t.sum(t.abs(self.model.l1_weight))

        # L2/L1 regularization for thetas
        for i in range(len(self.stack_types)):
            for block_id in range(self.n_blocks[i]):
                for layer in self.model.blocks[i].modules():
                    if isinstance(layer, t.nn.Linear):
                        # loss += self.lambda_reg_theta * t.norm(layer.weight)
                        loss += self.lambda_l1_theta * layer.weight.abs().sum()
        return loss

    def to_tensor(self, x: np.ndarray) -> t.Tensor:
        tensor = t.as_tensor(x, dtype=t.float32).to(self.device)
        return tensor

    def predict(self, ts_loader, eval_mode=False):
        self.model.eval()
        
        forecasts = []
        outsample_ys = []
        with t.no_grad():
            for batch in iter(ts_loader):
                insample_y     = self.to_tensor(batch['insample_y'])
                insample_x     = self.to_tensor(batch['insample_x'])
                insample_mask  = self.to_tensor(batch['insample_mask'])
                outsample_x    = self.to_tensor(batch['outsample_x'])
                outsample_y    = self.to_tensor(batch['outsample_y'])
                outsample_mask = self.to_tensor(batch['outsample_mask'])
                s_matrix       = self.to_tensor(batch['s_matrix'])

                forecast = self.model(insample_y=insample_y, insample_x_t=insample_x,
                                      insample_mask=insample_mask, outsample_x_t=outsample_x, x_s=s_matrix)
                
                forecasts += [forecast.cpu().data.numpy()]
                outsample_ys += [outsample_y.cpu().data.numpy()]

        forecasts = np.vstack(forecasts)
        outsample_ys = np.vstack(outsample_ys)

        if self.scaler is not None:
            if len(outsample_ys.shape) == 1:
                forecasts = forecasts.reshape(-1, 1)
                outsample_ys = outsample_ys.reshape(-1, 1)
            forecasts = self.scaler.inverse_transform(forecasts)
            if eval_mode: # Otherwise outsample_ys is None from batch
                outsample_ys = self.scaler.inverse_transform(outsample_ys)

        self.model.train()
        if eval_mode:
            return forecasts, outsample_ys
        else:
            return forecasts

    def evaluate_performance(self, ts_loader, validation_loss_fn):
        y_hat, y = self.predict(ts_loader=ts_loader, eval_mode=True)

        y = y.flatten()
        y_hat = y_hat.flatten()

        loss = validation_loss_fn(target=y, forecast=y_hat, weights=np.ones(len(y)))
        return loss

    def fit(self, train_ts_loader, val_ts_loader=None, max_epochs=None, verbose=True, eval_steps=1):
        # Asserts
        assert self.t_cols[0] == 'y', f'First variable must be y not {self.t_cols[0]}'
        #assert (self.input_size)==train_ts_loader.input_size, \
        #    f'model input_size {self.input_size} data input_size {train_ts_loader.input_size}'

        # Random Seeds (model initialization)
        t.manual_seed(self.random_seed)
        np.random.seed(self.random_seed)
        #random.seed(self.random_seed) #TODO: interaccion rara con window_sampling de validacion

        # Instantiate model
        #if not self._is_instantiated:
        block_list = self.create_stack()
        self.model = NBeats(blocks=t.nn.ModuleList(block_list), in_features=self.n_x_t).to(self.device)
        self._is_instantiated = True

        # Overwrite max_epochs and train datasets
        if max_epochs is None:
            max_epochs = self.max_epochs

        lr_decay_step_size = max_epochs // self.n_lr_decay_steps

        #optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
        optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=lr_decay_step_size, gamma=self.lr_decay)

        training_loss_fn = self.__loss_fn(self.loss)
        validation_loss_fn = self.__val_loss_fn(self.loss) #Uses numpy losses

        if verbose and (max_epochs > 0):
            print('='*30+' Start fitting '+'='*30)
            print(f'Number of exogenous variables: {self.n_x_t}')
            print(f'Number of static variables: {self.n_x_s} , with dim_hidden: {self.n_s_hidden}')
            print(f'Number of iterations: {max_epochs}')
            print(f'Number of blocks: {len(self.model.blocks)}')

        #self.loss_dict = {} # Restart self.loss_dict
        start = time.time()
        self.trajectories = {'step':[],'train_loss':[], 'val_loss':[]}
        self.final_insample_loss = None
        self.final_outsample_loss = None

        # Training Loop
        best_val_loss = np.inf
        best_insample_loss = np.inf
        break_flag = False
        early_stopping_counter = 0
        # Save initial weights as best_state_dict
        best_state_dict = copy.deepcopy(self.model.state_dict())
        for step in range(max_epochs):
            self.model.train()
            for batch in iter(train_ts_loader):
                insample_y     = self.to_tensor(batch['insample_y'])
                insample_x     = self.to_tensor(batch['insample_x'])
                insample_mask  = self.to_tensor(batch['insample_mask'])
                outsample_x    = self.to_tensor(batch['outsample_x'])
                outsample_y    = self.to_tensor(batch['outsample_y'])
                outsample_mask = self.to_tensor(batch['outsample_mask'])
                s_matrix       = self.to_tensor(batch['s_matrix'])

                optimizer.zero_grad()
                forecast = self.model(insample_y=insample_y, insample_x_t=insample_x,
                                    insample_mask=insample_mask, outsample_x_t=outsample_x, x_s=s_matrix)

                training_loss = training_loss_fn(x=insample_y, freq=self.seasonality, forecast=forecast,
                                                target=outsample_y, mask=outsample_mask)

                training_loss.backward()
                t.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                optimizer.step()

            lr_scheduler.step()
            if (step % eval_steps == 0):
                display_string = 'Step: {}, Time: {:03.3f}, Insample {}: {:.5f}'.format(step,
                                                                                time.time()-start,
                                                                                self.loss,
                                                                                training_loss.cpu().data.numpy())
                self.trajectories['step'].append(step)
                self.trajectories['train_loss'].append(training_loss.cpu().data.numpy())

                if val_ts_loader is not None:
                    loss = self.evaluate_performance(val_ts_loader, validation_loss_fn=validation_loss_fn)
                    display_string += ", Outsample {}: {:.5f}".format(self.loss, loss)
                    self.trajectories['val_loss'].append(loss)

                    if self.early_stopping:
                        if loss < best_val_loss:
                            # Save current model if improves outsample loss
                            best_state_dict = copy.deepcopy(self.model.state_dict())
                            best_insample_loss = training_loss.cpu().data.numpy()
                            early_stopping_counter = 0
                            best_val_loss = loss
                        else:
                            early_stopping_counter += 1
                        if early_stopping_counter >= self.early_stopping:
                            break_flag = True

                print(display_string)

                self.model.train()
                #train_ts_loader.train()

            if break_flag:
                print("\n")
                print(17*'-',' Stopped training with early stopping', 17*'-')
                self.model.load_state_dict(best_state_dict)
                break

        #End of fitting
        if max_epochs > 0:
            self.final_insample_loss = training_loss.cpu().data.numpy() if not break_flag else best_insample_loss #This is batch!
            string = 'Step: {}, Time: {:03.3f}, Insample {}: {:.5f}'.format(step,
                                                                            time.time()-start,
                                                                            self.loss,
                                                                            self.final_insample_loss)
            if val_ts_loader is not None:
                self.final_outsample_loss = self.evaluate_performance(val_ts_loader, validation_loss_fn=validation_loss_fn)
                string += ", Outsample {}: {:.5f}".format(self.loss, self.final_outsample_loss)
            print(string)
            print('='*30+' End  fitting '+'='*30)
            print("\n")

    def save(self, model_dir, model_id):

        if not os.path.exists(model_dir):
            os.makedirs(model_dir)

        model_file = os.path.join(model_dir, f"model_{model_id}.model")
        print('Saving model to:\n {}'.format(model_file)+'\n')
        t.save({'model_state_dict': self.model.state_dict()}, model_file)

    def load(self, model_dir, model_id):

        model_file = os.path.join(model_dir, f"model_{model_id}.model")
        path = Path(model_file)

        assert path.is_file(), 'No model_*.model file found in this path!'

        print('Loading model from:\n {}'.format(model_file)+'\n')

        checkpoint = t.load(model_file, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.to(self.device)


    def recalibrate_and_forecast_next_day(self, mc, df, next_day_date):
        """Method that builds an easy-to-use interface for daily recalibration and forecasting of the DNN model
        
        The method receives a pandas dataframe ``df`` and a day ``next_day_date``. Then, it 
        recalibrates the model using data up to the day before ``next_day_date`` and makes a prediction
        for day ``next_day_date``.
        
        Parameters
        ----------
        df : pandas.DataFrame
            Dataframe of historical data containing prices and N exogenous inputs. The index of the 
            dataframe should be dates with hourly frequency. The columns should have the following 
            names ['Price', 'Exogenous 1', 'Exogenous 2', ...., 'Exogenous N']
        next_day_date : TYPE
            Date of the day-ahead
        
        Returns
        -------
        numpy.array
            An array containing the predictions in the provided date
        
        """

        #---------------------------------------- Parse data ----------------------------------------#

        # We define the new training dataset considering the last calibration_window years of data 
        df_train = df.loc[:next_day_date - pd.Timedelta(hours=1)]
        df_train = df_train.loc[next_day_date - pd.Timedelta(hours=mc['calibration_window'] * 364 * 24):]

        # We define the test dataset as the next day (they day of interest) plus the last two weeks
        # in order to be able to build the necessary input features.
        df_test = df.loc[next_day_date - pd.Timedelta(weeks=2):, :]

        # Generating training, validation, and test input and outpus. For the test dataset,
        # even though the dataframe contains 15 days of data (next day + last 2 weeks),
        # we provide as parameter the date of interest so that Xtest and Ytest only reflect that
        Xtrain, Ytrain, Xval, Yval, Xtest, _, _ = \
            _build_and_split_XYs(dfTrain=df_train, features=mc, percentage_val=0.25,
                                 shuffle_train=True, dfTest=df_test, date_test=next_day_date,
                                 data_augmentation=mc['data_augmentation'], 
                                 n_exogenous_inputs=len(df_train.columns) - 1)

        #---------------------------------------- Scale data ----------------------------------------#
        # If required, datasets are scaled
        if mc['scaleX'] in ['Norm', 'Norm1', 'Std', 'Median', 'Invariant']:
            [Xtrain, Xval, Xtest], _ = scaling([Xtrain, Xval, Xtest], mc['scaleX'])

        if mc['scaleY'] in ['Norm', 'Norm1', 'Std', 'Median', 'Invariant']:
            [Ytrain, Yval], self.scaler = scaling([Ytrain, Yval], mc['scaleY'])
        else:
            self.scaler = None

        train_dataset = TimeSeriesDataset(X=Xtrain, Y=Ytrain)
        train_loader = DataLoader(train_dataset, batch_size=mc['batch_size'], shuffle=True, num_workers=0)

        val_dataset = TimeSeriesDataset(X=Xval, Y=Yval)
        val_loader = DataLoader(val_dataset, batch_size=len(Xval), shuffle=False, num_workers=0)

        test_dataset = TimeSeriesDataset(X=Xtest, Y=None)
        test_loader = DataLoader(test_dataset, batch_size=len(Xtest), shuffle=False, num_workers=0)

        #---------------------------------------- Recalibrate ----------------------------------------#
        # Initialize and fit model
        self.fit(train_loader, val_ts_loader=val_loader, verbose=True, eval_steps=mc['eval_steps']) 

        #---------------------------------------- Predict ----------------------------------------#
        # Make prediction
        y_hat = self.predict(ts_loader=test_loader, eval_mode=False)

        return y_hat

In [5]:
def print_model(model):
    print(120*'=')
    line = '{:<40} {:<40} {:<40}'.format('Weights', 'Parameter shape', 'Number of parameters')
    print(line)
    print(120*'-')
    total_params = 0
    for model_param_name, model_param_value in model.model.named_parameters():
        line = '{:<50} {:<40} {:<40}'.format(model_param_name, str(model_param_value.shape), len(model_param_value.flatten()))
        total_params += len(model_param_value.flatten())
        print(line)
        print(120*'-')
    print(f'Total parameters: {total_params}')
    print(120*'=')
    print(model.model)

In [6]:
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader

class TimeSeriesDataset(Dataset):
    def __init__(self, X, Y=None):
        """
        """        
        self.insample_y  = X[:, (7*0)*24:(7*1)*24]

        insample_x1 = X[:, (7*1)*24:(7*2)*24]
        insample_x2 = X[:, (7*2)*24:(7*3)*24]
        insample_x3 = X[:, (7*3)*24:(7*4)*24]
        
        self.insample_x  = np.concatenate([insample_x1[:,None,:], 
                                           insample_x2[:,None,:], 
                                           insample_x3[:,None,:]], axis=1)

        self.insample_mask = np.ones(shape=self.insample_y.shape)
        
        outsample_x1 = X[:, (7*4+0)*24:(7*4+1)*24]
        outsample_x2 = X[:, (7*4+1)*24:(7*4+2)*24]
        outsample_x3 = X[:, (7*4+2)*24:(7*4+3)*24]
        
        self.outsample_x  = np.concatenate([outsample_x1[:,None,:], 
                                            outsample_x2[:,None,:], 
                                            outsample_x3[:,None,:]], axis=1)

        self.outsample_y = Y
        self.outsample_mask = np.ones(shape=(self.outsample_x.shape[0], self.outsample_x.shape[2]))

        assert np.sum(np.isnan(X))<1.0, \
            f' X has {np.sum(np.isnan(X))} nan values'

        if Y is not None:
            assert np.sum(np.isnan(Y))<1.0, \
                f'Y has {np.sum(np.isnan(Y))} nan values'

    def __len__(self):
        return len(self.outsample_x)

    def __getitem__(self, idx):        
        # insample y
        insample_y     = self.insample_y[idx, :]

        # insample and outsample x
        insample_x     = self.insample_x[idx, :, :]
        outsample_x    = self.outsample_x[idx, :, :]
        
        # train masks
        insample_mask  = self.insample_mask[idx, :]
        outsample_mask = self.outsample_mask[idx, :]

        # outsample y
        if self.outsample_y is not None:
            outsample_y    = self.outsample_y[idx, :]
        else:
            outsample_y    = []

        batch = {'s_matrix': [],
                 'insample_y': insample_y, 'insample_x':insample_x, 'insample_mask':insample_mask,
                 'outsample_y': outsample_y, 'outsample_x':outsample_x, 'outsample_mask':outsample_mask}
        return batch


In [10]:
import time
import numpy as np
import pandas as pd
import random
import torch as t
import copy
from fastcore.foundation import patch
from nixtla.data.scalers import scaling
# from nixtla.data.ontsdataset import TimeSeriesDataset
# from nixtla.data.ontsloader_fast import TimeSeriesLoader as TimeSeriesLoaderFast

np.random.seed(1)
t.manual_seed(1)

Xtrain = np.genfromtxt('../data/epf/Xtrain.csv')
Ytrain = np.genfromtxt('../data/epf/Ytrain.csv')

[Xtrain], _ = scaling(datasets=[Xtrain], normalize='Norm1')
[Y_train], scaler = scaling(datasets=[Ytrain], normalize='Norm1')

print('X: time series features, of shape (#series,#times,#features): \t' + str(Xtrain.shape))
print('Y: target series (in X), of shape (#series,#times): \t \t' + str(Ytrain.shape))
# print('S: static features, of shape (#series,#features): \t \t' + str(S.shape))
#Y_insample_df.head()
#Y_insample_df.tail()
print("\n\n")

train_dataset = TimeSeriesDataset(X=Xtrain, Y=Ytrain)
train_loader = DataLoader(train_dataset, batch_size=4,
                          shuffle=True, num_workers=0)

val_dataset = TimeSeriesDataset(X=Xtrain, Y=Ytrain)
val_loader = DataLoader(val_dataset, batch_size=4,
                        shuffle=True, num_workers=0)                        

start = time.time()
dataloader = iter(train_loader)
batch = next(dataloader)
insample_y = batch['insample_y']
insample_x = batch['insample_x']
insample_mask = batch['insample_mask']
outsample_x = batch['outsample_x']
outsample_y = batch['outsample_y']
outsample_mask = batch['outsample_mask']

print("DataloaderFast batch time:", time.time()-start)
print("insample_y.shape", insample_y.shape)
print("insample_x.shape", insample_x.shape)
print("insample_mask.shape", insample_mask.shape)

print("outsample_y.shape", outsample_y.shape)
print("outsample_x.shape", outsample_x.shape)
print("outsample_mask.shape", outsample_mask.shape)

print("t.max(insample_y)", t.max(insample_y))
print("t.max(outsample_y)", t.max(outsample_y * outsample_mask))
print("\n\n")

X: time series features, of shape (#series,#times,#features): 	(1087, 744)
Y: target series (in X), of shape (#series,#times): 	 	(1087, 24)



DataloaderFast batch time: 0.001127004623413086
insample_y.shape torch.Size([4, 168])
insample_x.shape torch.Size([4, 3, 168])
insample_mask.shape torch.Size([4, 168])
outsample_y.shape torch.Size([4, 24])
outsample_x.shape torch.Size([4, 3, 24])
outsample_mask.shape torch.Size([4, 24])
t.max(insample_y) tensor(0.2681, dtype=torch.float64)
t.max(outsample_y) tensor(35.5800, dtype=torch.float64)





In [11]:
nbeatsx = Nbeats(n_x_t=3,
                 n_x_s=0,
                 t_cols=['y', 'Exogenous1', 'Exogenous2', 'week_day'],
                 include_var_dict={'y': [-2, -8], 
                                   'Exogenous1': [-2, -8],
                                   'Exogenous2': [-2, -8],
                                   'week_day': [-1]},
                 input_size_multiplier=7,
                 output_size=24,
                 shared_weights=False,
                 activation='relu',
                 initialization='lecun_normal',                
                 stack_types=['exogenous_wavenet']+3*['identity'],
                 n_blocks=4*[1],
                 n_theta_hidden_list=4*[[256,256]],
                 n_xbasis_layers=4,
                 n_xbasis_channels=2,
                 n_s_hidden=10,
                 scaler=scaler, # It is updated in recalibrate_and_forecast_next_day method
                 batch_normalization=False,
                 learning_rate=0.001,
                 lr_decay=0.5,
                 n_lr_decay_steps=3,
                 dropout_prob_theta=0.1,
                 dropout_prob_xbasis=0.1,
                 lambda_l1_input=0.01,
                 lambda_l1_theta=0.01,
                 lambda_l2_xbasis=0.01,
                 max_epochs=100,
                 early_stopping=20,
                 loss='MAE',
                 frequency=24,
                 seasonality='H',
                 random_seed=1)

nbeatsx.fit(train_ts_loader=train_loader, val_ts_loader=val_loader, max_epochs=20, verbose=True, eval_steps=1)

============================== Start fitting ==============================
Number of exogenous variables: 3
Number of static variables: 0 , with dim_hidden: 10
Number of iterations: 20
Number of blocks: 4
Step: 0, Time: 3.722, Insample MAE: 5.18549, Outsample MAE: 139.05465
Step: 1, Time: 8.133, Insample MAE: 4.82959, Outsample MAE: 114.44603
Step: 2, Time: 12.571, Insample MAE: 4.15717, Outsample MAE: 122.55745
Step: 3, Time: 16.998, Insample MAE: 5.20711, Outsample MAE: 103.47394
Step: 4, Time: 21.431, Insample MAE: 4.87293, Outsample MAE: 103.24238
Step: 5, Time: 25.856, Insample MAE: 4.04834, Outsample MAE: 99.55143
Step: 6, Time: 31.311, Insample MAE: 4.44104, Outsample MAE: 102.26979
Step: 7, Time: 36.075, Insample MAE: 4.15949, Outsample MAE: 102.07084
Step: 8, Time: 40.512, Insample MAE: 4.87403, Outsample MAE: 104.04875
Step: 9, Time: 45.148, Insample MAE: 3.43742, Outsample MAE: 97.02498
Step: 10, Time: 49.568, Insample MAE: 4.34725, Outsample MAE: 102.09178
Step: 11, Time: 

In [ ]:
train_loader.datas